preprocessing method in kilosort 4 
TODO


In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

os.chdir('/Users/xiao-zy19/Desktop/good_cs')
data = np.fromfile('Good_cplx_spikes.bin', dtype=np.int16).reshape(-1, 128) #reshape((-1, 128))
print(data.shape)

(6122790, 128)


In [3]:
# from scipy.signal import butter, filtfilt

# fs = 30000
# lowcut = 300

# b, a = butter(3, 300, fs=fs, btype='high')
# data_filtered = filtfilt(b, a, data, axis=0)

# plt.plot(data_filtered[:1000, 0])

In [ ]:
from scipy.signal import butter, filtfilt

# fs = 30000
# lowcut = 300
# NT = 30122

# b, a = butter(3, 300, fs=fs, btype='high')
# x = np.zeros(NT)
# x[NT//2] = 1
# hp_filter = filtfilt(b, a, x).copy()

NT = 60000
nt = 61
n_samples = data.shape[0]
n_batches = int(np.ceil(n_samples / NT))

def get_highpass_filter(fs=30000):
    NT = 30122
    # a butterworth 300hz highpass filter
    b,a = butter(3, 300, fs = fs, btype = 'high')
    # compute the impulse response
    x = np.zeros(NT)
    x[NT//2] = 1
    # symmetric filter from scipy
    hp_filter = filtfilt(b, a , x).copy()
    return hp_filter

def fft_highpass(hp_filter, NT=30122):
    ft = hp_filter.shape[0]
    
    if ft < NT:
        pad = (NT - ft) // 2
        fhp = np.fft.fft(np.concatenate((np.zeros(pad), hp_filter, np.zeros(pad + (NT-pad*2-ft)))))
    elif ft > NT:
        crop = (ft - NT) // 2
        fhp = np.fft.fft(hp_filter[crop : crop + NT])
    else:
        fhp = np.fft.fft(hp_filter)
    return fhp

def pad_data(X, ibatch):
    
    

def filter(X, hp_filter):
    # high-pass filtering in the Fourier domain (faster than filtfilt)
    fwav = fft_highpass(hp_filter, NT=X.shape[1])
    X = np.real(np.fft.ifft(np.fft.fft(X) * np.conj(fwav)))
    X = np.fft.fftshift(X, axes = -1)
    return X

def get_whiten_mat(X, nskip=25, nrange=None):
    n_chan = X.shape[0]
    # collect the covariance matrix across channels
    CC = np.zeros((n_chan, n_chan))
    k = 0
    for j in range(0, X.shape[1]-1, nskip):
        # load data with high-pass filtering (see the Binary file class)
        # X = f.padded_batch_to_torch(j)        
        # remove padding
        X = X[:, j : -j]
        # cumulative covariance matrix
        CC = CC + (X @ X.T)/X.shape[1]
        k+=1
    CC = CC / k
    
    # compute the local whitening filters and collect back into Wrot
    if nrange is None:
        Wrot = whitening_local(CC, nrange=n_chan)
    else:
        Wrot = whitening_local(CC, nrange=nrange)

    return Wrot

def whitening_local(CC, nrange=32):
    """ loop through each channel and compute its whitening filter based on nearest channels
    """
    Nchan = CC.shape[0]
    Wrot = np.zeros((Nchan,Nchan))

    # for each channel, a local covariance matrix is extracted
    # the whitening matrix is computed for that local neighborhood
    for j in range(CC.shape[0]):
        ds = np.arange(Nchan)
        isort = np.argsort(ds)
        ix = isort[:nrange]

        wrot = whitening_from_covariance(CC[np.ix_(ix, ix)])

        # the first row of wrot is a whitening vector for the center channel
        Wrot[j, ix] = wrot[0]
    return Wrot


# def get_whitening_matrix(f, xc, yc, nskip=25, nrange=32):
#     """ get the whitening matrix, use every nskip batches
#     """
#     n_chan = len(f.chan_map)
#     # collect the covariance matrix across channels
#     CC = torch.zeros((n_chan, n_chan), device=f.device)
#     k = 0
#     for j in range(0, f.n_batches-1, nskip):
#         # load data with high-pass filtering (see the Binary file class)
#         X = f.padded_batch_to_torch(j)        
        
#         # remove padding
#         X = X[:, f.nt : -f.nt]

#         # cumulative covariance matrix
#         CC = CC + (X @ X.T)/X.shape[1]
        
#         k+=1
        
#     CC = CC / k

#     # compute the local whitening filters and collect back into Wrot
#     Wrot = whitening_local(CC, xc, yc, nrange=nrange, device=f.device)

#     return Wrot

# def whitening_from_covariance(CC):
#     """ whitening matrix for a covariance matrix CC
#         this is the so-called ZCA whitening matrix
#     """
#     E,D,V =  torch.linalg.svd(CC)
#     eps = 1e-6
#     Wrot =(E / (D+eps)**.5) @ E.T
#     return Wrot

# def whitening_local(CC, xc, yc, nrange=32, device=torch.device('cuda')):
#     """ loop through each channel and compute its whitening filter based on nearest channels
#     """
#     Nchan = CC.shape[0]
#     Wrot = torch.zeros((Nchan,Nchan), device = device)

#     # for each channel, a local covariance matrix is extracted
#     # the whitening matrix is computed for that local neighborhood
#     for j in range(CC.shape[0]):
#         ds = (xc[j] - xc)**2 + (yc[j] - yc)**2
#         isort = np.argsort(ds)
#         ix = isort[:nrange]

#         wrot = whitening_from_covariance(CC[np.ix_(ix, ix)])

#         # the first row of wrot is a whitening vector for the center channel
#         Wrot[j, ix] = wrot[0]
#     return Wrot

